## Table of content

* Distribution shapes

* * Examples

* * Reshaping distributions

* * It is always safe to assume dependence

* Declaring independence with plate

* Subsampling inside plate

* Broadcasting to allow Parallel Enumeration

* * Writing parallelizable code

* * Automatic broadcasting inside pyro.plate


In [1]:
import os
import torch
import pyro
from torch.distributions import constraints
from pyro.distributions import Bernoulli, Categorical, MultivariateNormal, Normal
from pyro.distributions.util import broadcast_shape
from pyro.infer import Trace_ELBO, TraceEnum_ELBO, config_enumerate
import pyro.poutine as poutine
from pyro.optim import Adam

smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.4')

# we'll use this helper to check our models are correct
def test_model(model, guide, loss):
    pyro.clear_param_store()
    loss.loss(model, guide)

/Users/sina/Library/Mobile Documents/com~apple~CloudDocs/Git_Projects/Deep-probabilistic-modeling/PyroVenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Distributions shapes: batch_shape and event_shape

PyTorch Tensors have a single .shape attribute, but Distributions have two shape attributions with special meaning: .batch_shape and .event_shape. These two combine to define the total shape of a sample.

``` python
x = d.sample()
assert x.shape == d.batch_shape + d.event_shape

```

Indices over .batch_shape denote conditionally independent random variables, whereas indices over .event_shape denote dependent random variables (ie one draw from a distribution). Because the dependent random variables define probability together, the .log_prob() method only produces a single number for each event of shape .event_shape. Thus the total shape of .log_prob() is .batch_shape:

```python
assert d.log_prob(x).shape == d.batch_shape
```

Note that the Distribution.sample() method also takes a sample_shape parameter that indexes over independent identically distributed (iid) random varables, so that:

```python
x2 = d.sample(sample_shape)
assert x2.shape == sample_shape + batch_shape + event_shape

```

in summary:

```
      |      iid     | independent | dependent
------+--------------+-------------+------------
shape = sample_shape + batch_shape + event_shape

```

For example univariate distributions have empty event shape (because each number is an independent event). Distributions over vectors like MultivariateNormal have len(event_shape) == 1. Distributions over matrices like InverseWishart have len(event_shape) == 2.

### Examples
The simplest distribution shape is a single univariate distribution.


In [2]:
d = Bernoulli(0.5)
assert d.batch_shape == ()
assert d.event_shape == ()
x = d.sample()
assert x.shape == ()
assert d.log_prob(x).shape == ()

In [3]:
'''
Distributions can be batched by passing in batched parameters.
'''

d = Bernoulli(0.5 * torch.ones(3, 4))
assert d.batch_shape == (3, 4)
assert d.event_shape == ()
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3, 4)

In [4]:
'''
Another way to batch distributions is via the .expand() method. This only works if parameters are identical along the leftmost dimensions.
'''

d = Bernoulli(torch.tensor([0.1, 0.2, 0.3, 0.4])).expand([3, 4])
assert d.batch_shape == (3, 4)
assert d.event_shape == ()
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3, 4)

In [5]:
'''
Multivariate distributions have nonempty .event_shape. For these distributions, the shapes of .sample() and .log_prob(x) differ:
'''

d = MultivariateNormal(torch.zeros(3), torch.eye(3, 3))
assert d.batch_shape == ()
assert d.event_shape == (3, )
x = d.sample()
assert x.shape == (3, )             # == batch_shape + even_shape 
assert d.log_prob(x).shape == ()    # == batch shape

### Reshaping distributions

In Pyro you can treat a univariate distribution as multivariate by calling the .to_event(n) property where n is the number of batch dimensions (from the right) to declare as dependent.

In [8]:
d = Bernoulli(0.5 * torch.ones(3, 4)).to_event(1)
assert d.batch_shape == (3, )
assert d.event_shape == (4, )
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3, )

'''
While you work with Pyro programs, keep in mind that samples have
shape batch_shape + event_shape, whereas .log_prob(x) values have
shape batch_shape. You’ll need to ensure that batch_shape is carefully
controlled by either trimming it down with .to_event(n) or by declaring
dimensions as independent via pyro.plate.
'''

'\nWhile you work with Pyro programs, keep in mind that samples have\nshape batch_shape + event_shape, whereas .log_prob(x) values have\nshape batch_shape. You’ll need to ensure that batch_shape is carefully\ncontrolled by either trimming it down with .to_event(n) or by declaring\ndimensions as independent via pyro.plate.\n'

### It is always safe to assume dependence

Often in Pyro we’ll declare some dimensions as dependent even though they are in fact independent, e.g.

``` python
x = pyro.sample("x", Normal(0, 1).expand([10]).to_event(1))
assert x.shape == (10,)
```

This is useful for two reasons: First it allows us to easily swap in a MultivariateNormal distribution later. Second it simplifies the code a bit since we don’t need a plate (see below) as in

``` python
with pyro.plate("x_plate", 10):
    x = pyro.sample("x", Normal(0, 1))  # .expand([10]) is automatic
    assert x.shape == (10,)
```

The difference between these two versions is that the second version with plate informs Pyro that it can make use of conditional independence information when estimating gradients, whereas in the first version Pyro must assume they are dependent (even though the normals are in fact conditionally independent). This is analogous to d-separation in graphical models: it is always safe to add edges and assume variables may be dependent (i.e. to widen the model class), but it is unsafe to assume independence when variables are actually dependent (i.e. narrowing the model class so the true model lies outside of the class, as in mean field). In practice Pyro’s SVI inference algorithm uses reparameterized gradient estimators for Normal distributions so both gradient estimators have the same performance.

### Declaring independent dims with plate
Pyro models can use the context manager pyro.plate to declare that certain batch dimensions are independent. Inference algorithms can then take advantage of this independence to e.g. construct lower variance gradient estimators or to enumerate in linear space rather than exponential space. An example of an independent dimension is the index over data in a minibatch: each datum should be independent of all others.

The simplest way to declare a dimension as independent is to declare the rightmost batch dimension as independent via a simple

```python
with pyro.plate("my_plate"):
    # within this context, batch dimension -1 is independent
```

We recommend always providing an optional size argument to aid in debugging shapes

```python
with pyro.plate("my_plate", len(my_data)):
    # within this context, batch dimension -1 is independent
```

Starting with Pyro 0.2 you can additionally nest plates, e.g. if you have per-pixel independence:

```python
with pyro.plate("x_axis", 320):
    # within this context, batch dimension -1 is independent
    with pyro.plate("y_axis", 200):
        # within this context, batch dimensions -2 and -1 are independent

```

Note that we always count from the right by using negative indices like -2, -1.

Finally if you want to mix and match plates for e.g. noise that depends only on x, some noise that depends only on y, and some noise that depends on both, you can declare multiple plates and use them as reusable context managers. In this case Pyro cannot automatically allocate a dimension, so you need to provide a dim argument (again counting from the right):

```python
x_axis = pyro.plate("x_axis", 3, dim=-2)
y_axis = pyro.plate("y_axis", 2, dim=-3)
with x_axis:
    # within this context, batch dimension -2 is independent
with y_axis:
    # within this context, batch dimension -3 is independent
with x_axis, y_axis:
    # within this context, batch dimensions -3 and -2 are independent
```

In [9]:
'''
let's take a closer look at batch sizes within "plate" s.
'''

def model1():
    a = pyro.sample("a", Normal(0, 1))
    b = pyro.sample("b", Normal(torch.zeros(2), 1).to_event(1))
    with pyro.plate("c_plate", 2):
        c = pyro.sample("c", Normal(torch.zeros(2), 1))
    with pyro.plate("d_plate", 3):
        d = pyro.sample("d", Normal(torch.zeros(3, 4, 5), 1).to_event(2))
    assert a.shape == ()         # batch_shape == ()    event_shape == ()
    assert b.shape == (2, )      # batch_shape == ()    event_shape == (2, )
    assert c.shape == (2, )      # batch_shape == (2, ) event_shape == ()
    assert d.shape == (3, 4, 5)  # batch_shape == (3, ) event_shape == (4, 5)

    x_axis = pyro.plate("x_axis", 3, dim=-2)
    y_axis = pyro.plate("y_axis", 2, dim=-3)
    with x_axis:
        x = pyro.sample("x", Normal(0, 1))
    with y_axis:
        y = pyro.sample("y", Normal(0, 1))
    with x_axis, y_axis:
        xy = pyro.sample("xy", Normal(0, 1))
        z = pyro.sample("z", Normal(0, 1).expand([5]).to_event(1))
    assert x.shape == (3, 1)       # batch_shape == (3, 1)     event_shape == ()
    assert y.shape == (2, 1, 1)    # batch_shaoe == (2, 1, 1)   event_shape == ()
    assert xy.shape == (2, 3, 1)   # batch__shape == (2, 3, 1)   event_shape == ()
    assert z.shape ==(2, 3, 1, 5)  # batch_shaoe == (2, 3, 1)    event_shape == (5, )

test_model(model1, model1, Trace_ELBO())

It is helpful to visualize the .shapes of each sample site by aligning them at the boundary between batch_shape and event_shape: dimensions to the right will be summed out in .log_prob() and dimensions to the left will remain.

```python
batch dims | event dims
-----------+-----------
           |        a = sample("a", Normal(0, 1))
           |2       b = sample("b", Normal(zeros(2), 1)
           |                        .to_event(1))
           |        with plate("c", 2):
          2|            c = sample("c", Normal(zeros(2), 1))
           |        with plate("d", 3):
          3|4 5         d = sample("d", Normal(zeros(3,4,5), 1)
           |                       .to_event(2))
           |
           |        x_axis = plate("x", 3, dim=-2)
           |        y_axis = plate("y", 2, dim=-3)
           |        with x_axis:
        3 1|            x = sample("x", Normal(0, 1))
           |        with y_axis:
      2 1 1|            y = sample("y", Normal(0, 1))
           |        with x_axis, y_axis:
      2 3 1|            xy = sample("xy", Normal(0, 1))
      2 3 1|5           z = sample("z", Normal(0, 1).expand([5])
           |                       .to_event(1))
```

To examine the shapes of sample sites in a program automatically, you can trace the program and use the Trace.format_shapes() method, which prints three shapes for each sample site: the distribution shape (both site["fn"].batch_shape and site["fn"].event_shape), the value shape (site["value"].shape), and if log probability has been computed also the log_prob shape (site["log_prob"].shape):



In [10]:
trace = poutine.trace(model1).get_trace()
trace.compute_log_prob()  # optional, but allows printing of log_prob shapes
print(trace.format_shapes())

Trace Shapes:            
 Param Sites:            
Sample Sites:            
       a dist       |    
        value       |    
     log_prob       |    
       b dist       | 2  
        value       | 2  
     log_prob       |    
 c_plate dist       |    
        value     2 |    
     log_prob       |    
       c dist     2 |    
        value     2 |    
     log_prob     2 |    
 d_plate dist       |    
        value     3 |    
     log_prob       |    
       d dist     3 | 4 5
        value     3 | 4 5
     log_prob     3 |    
  x_axis dist       |    
        value     3 |    
     log_prob       |    
  y_axis dist       |    
        value     2 |    
     log_prob       |    
       x dist   3 1 |    
        value   3 1 |    
     log_prob   3 1 |    
       y dist 2 1 1 |    
        value 2 1 1 |    
     log_prob 2 1 1 |    
      xy dist 2 3 1 |    
        value 2 3 1 |    
     log_prob 2 3 1 |    
       z dist 2 3 1 | 5  
        value 2 3 1 | 5  
     log_pro